# Asistente de soporte inteligente para una empresa de tecnología

Una empresa de software llamada TechNova tiene miles de tickets de soporte de sus clientes acumulados en su sistema.

Cada ticket contiene:

- El texto de la consulta del cliente.

- El tipo de problema (categoría: instalación, conexión, errores, licencias, etc.).

- Y una respuesta técnica redactada por los agentes.

El departamento de soporte quiere crear un asistente interno inteligente que, al recibir una nueva consulta, recomiende los tickets anteriores más parecidos en significado (aunque usen palabras diferentes), para ayudar a los agentes a responder más rápido.

## Objetivo general

Desarrollar un programa en Python que:

1. Carga el archivo csv de tickets.

2. Genera embeddings para cada ticket (texto de problema).

3. Almacena en una base de datos vectorial.

4. Permite ingresar una nueva consulta del cliente.

5. Devuelve los tickets más similares en significado, junto con su respuesta original.

In [ ]:
#1.

In [ ]:
#2.

In [ ]:
#3.

In [ ]:
#4.

In [ ]:
#5.

## Herramientas y tecnologías sugeridas

Python 3.10+

VS Code

Librerías:


In [ ]:
pip install chromadb sentence-transformers pandas

## Preguntas

1. Diseño del vector

- ¿Qué campos del ticket deben incluirse en el texto del embedding (solo “problema” o también “categoría”)?

- ¿Cómo afectaría esto los resultados?

2. Elección de métrica

- ¿Por qué es mejor usar cosine similarity que distancia euclidiana para texto?

3. Optimización

- ¿Qué pasaría si hay 100 000 tickets?

- ¿Qué tipo de indexación sería más eficiente (HNSW, IVF, PQ)?

4. Interpretación

- Si los tres tickets más similares pertenecen a la misma categoría, ¿qué conclusión se puede sacar?

5. Ampliación (opcional)

- ¿Cómo integrarían este sistema en una interfaz web o chatbot?